# Подготовка данных

## Считаем данные в исходном csv формате

In [1]:
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable


Перезапустить kernel после установки!

In [1]:
import findspark
findspark.init()

In [6]:
!hdfs dfs -ls

Found 1 items
drwxr-xr-x   - ubuntu hadoop          0 2024-07-25 18:09 data


In [7]:
!hdfs dfs -ls data

Found 1 items
-rw-r--r--   1 ubuntu hadoop 5846760913 2024-07-25 18:09 data/train.csv


Создадим SparkSession

In [8]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName("OTUS")
        .getOrCreate()
)

Загрузим данные

In [9]:
df = spark.read.csv("data/train.csv", inferSchema=True, header=True)

Поделим данные на 10 разделов и сохраним в формате parquete

In [10]:
%%time
df.count()

CPU times: user 5.03 ms, sys: 530 µs, total: 5.56 ms
Wall time: 22.5 s


101230332

In [11]:
(
    df
        .repartition(10)
        .write
        .mode("overwrite")
        .parquet("data/train.parquet")
)

Проверим размер сохраненного файла

In [12]:
!hdfs dfs -ls -h data/train.parquet

Found 11 items
-rw-r--r--   1 ubuntu hadoop          0 2024-07-25 18:20 data/train.parquet/_SUCCESS
-rw-r--r--   1 ubuntu hadoop    181.0 M 2024-07-25 18:20 data/train.parquet/part-00000-ffcd8b6c-38f9-4c2e-8ccc-56e56aa4f540-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop    181.0 M 2024-07-25 18:20 data/train.parquet/part-00001-ffcd8b6c-38f9-4c2e-8ccc-56e56aa4f540-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop    181.2 M 2024-07-25 18:20 data/train.parquet/part-00002-ffcd8b6c-38f9-4c2e-8ccc-56e56aa4f540-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop    181.0 M 2024-07-25 18:20 data/train.parquet/part-00003-ffcd8b6c-38f9-4c2e-8ccc-56e56aa4f540-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop    181.0 M 2024-07-25 18:20 data/train.parquet/part-00004-ffcd8b6c-38f9-4c2e-8ccc-56e56aa4f540-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop    181.2 M 2024-07-25 18:20 data/train.parquet/part-00005-ffcd8b6c-38f9-4c2e-8ccc-56e56aa4f540-c000.snappy.parquet
-rw-r--r--   1 ubuntu hadoop    181.

Сравним скорость посчета строк из parquet

In [13]:
df_from_parquet = spark.read.parquet("data/train.parquet")

In [14]:
%%time
df_from_parquet.count()

CPU times: user 0 ns, sys: 2 ms, total: 2 ms
Wall time: 876 ms


101230332